In [1]:
import pandas as pd
import numpy as np
import os
import ipaddress
from pathlib import Path
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
pio.renderers.default = "notebook"

print("All libraries loaded successfully!")


All libraries loaded successfully!


In [2]:
# Load the split datasets 
print("Loading split datasets from split_data directory...")

# Load IPv4 datasets
df_ipv4_20250613 = pd.read_csv('split_data/ipv4_20250613.csv')
df_ipv4_20250923 = pd.read_csv('split_data/ipv4_20250923.csv')

# Load IPv6 datasets (aggregated to /64 prefixes)
df_ipv6_20250613 = pd.read_csv('split_data/ipv6_64_20250613.csv')
df_ipv6_20250923 = pd.read_csv('split_data/ipv6_64_20250923.csv')

print(f"IPv4 2025-06-13 shape: {df_ipv4_20250613.shape}")
print(f"IPv4 2025-09-23 shape: {df_ipv4_20250923.shape}")
print(f"IPv6 2025-06-13 shape: {df_ipv6_20250613.shape}")
print(f"IPv6 2025-09-23 shape: {df_ipv6_20250923.shape}")

print("\nExamining VPN service providers...")
print(f"IPv4 2025-06-13 unique services: {df_ipv4_20250613['service'].nunique()}")
print(f"IPv4 2025-09-23 unique services: {df_ipv4_20250923['service'].nunique()}")
print(f"IPv6 2025-06-13 unique services: {df_ipv6_20250613['service'].nunique()}")  
print(f"IPv6 2025-09-23 unique services: {df_ipv6_20250923['service'].nunique()}")


Loading split datasets from split_data directory...


/tmp/ipykernel_41444/550744705.py:5: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.

/tmp/ipykernel_41444/550744705.py:6: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



IPv4 2025-06-13 shape: (16450714, 8)
IPv4 2025-09-23 shape: (16899121, 8)
IPv6 2025-06-13 shape: (226166, 8)
IPv6 2025-09-23 shape: (253807, 8)

Examining VPN service providers...
IPv4 2025-06-13 unique services: 194
IPv4 2025-09-23 unique services: 214
IPv6 2025-06-13 unique services: 32
IPv6 2025-09-23 unique services: 35


In [12]:
# Create VPN service provider transition analysis function
def analyze_vpn_service_transitions(df_old, df_new, ip_version):
    """Analyze how VPN service providers change between time periods"""
    
    print(f"\n=== {ip_version} VPN Service Provider Transitions ===")
    
    # Set network as index
    df_old_indexed = df_old.set_index('network')
    df_new_indexed = df_new.set_index('network')
    
    # Get VPN networks only (vpn=True)
    vpn_old = df_old_indexed[df_old_indexed['vpn'] == True]
    vpn_new = df_new_indexed[df_new_indexed['vpn'] == True]
    
    print(f"VPN networks in 2025-06-13: {len(vpn_old):,}")
    print(f"VPN networks in 2025-09-23: {len(vpn_new):,}")
    
    # Find common VPN networks (those marked as VPN in both periods)
    common_vpn_networks = set(vpn_old.index) & set(vpn_new.index)
    print(f"Common VPN networks: {len(common_vpn_networks):,}")
    
    if len(common_vpn_networks) == 0:
        print("No common VPN networks found!")
        return None
        
    # Filter to common networks
    vpn_old_common = vpn_old.loc[list(common_vpn_networks)]
    vpn_new_common = vpn_new.loc[list(common_vpn_networks)]
    
    # Create transition matrix
    transitions = pd.DataFrame({
        'network': list(common_vpn_networks),
        'service_old': vpn_old_common['service'].values,
        'service_new': vpn_new_common['service'].values
    })
    
    # Clean up service names - replace NaN with 'Unknown/Generic VPN'
    transitions['service_old'] = transitions['service_old'].fillna('Unknown/Generic VPN')
    transitions['service_new'] = transitions['service_new'].fillna('Unknown/Generic VPN')
    
    # Count transitions
    transition_counts = transitions.groupby(['service_old', 'service_new']).size().reset_index(name='count')
    
    print(f"\nTotal service transitions tracked: {len(transition_counts)}")
    print(f"Top 10 transitions:")
    print(transition_counts.sort_values('count', ascending=False).head(10))
    
    return transition_counts

print("VPN transition analysis function defined!")


VPN transition analysis function defined!


In [40]:
# Create FAST Sankey diagram function - optimized for speed
def create_vpn_service_sankey(transitions_df, ip_version, min_count=50, top_n=None):
    
    if transitions_df is None or len(transitions_df) == 0:
        print(f"No data for {ip_version}")
        return
    
    print(f"Total transitions before filtering: {len(transitions_df)}")
    
    if top_n:
        data = transitions_df.nlargest(top_n, 'count')
    else:
        mask = transitions_df['count'] >= min_count
        data = transitions_df[mask] if mask.any() else transitions_df.nlargest(20, 'count')
    
    print(f"{ip_version}: {len(data)} transitions after filtering")
    
    # Speed optimization: direct list operations instead of iterative
    # Sort services by total transition count to order the nodes
    service_counts = {}
    for _, row in data.iterrows():
        service_counts[row['service_old']] = service_counts.get(row['service_old'], 0) + row['count']
        service_counts[row['service_new']] = service_counts.get(row['service_new'], 0) + row['count']
    
    # Sort services by their total count (descending)
    services = sorted(set(data['service_old']) | set(data['service_new']), 
                     key=lambda s: service_counts.get(s, 0), reverse=True)
    service_map = {s: i for i, s in enumerate(services)}
    n = len(services)
    
    # Vectorized operations
    source = [service_map[old] for old in data['service_old']]
    target = [service_map[new] + n for new in data['service_new']]
    value = data['count'].tolist()
    
    # Calculate node values for display
    old_counts = {}
    new_counts = {}
    for _, row in data.iterrows():
        old_counts[row['service_old']] = old_counts.get(row['service_old'], 0) + row['count']
        new_counts[row['service_new']] = new_counts.get(row['service_new'], 0) + row['count']
    
    # Labels with counts
    labels = [f"{s} ({old_counts.get(s, 0):,})" for s in services] + [f"{s} ({new_counts.get(s, 0):,})" for s in services]
    
    # Quick Sankey creation
    fig = go.Figure(go.Sankey(
        node=dict(pad=8, thickness=12, label=labels),
        link=dict(source=source, target=target, value=value)
    ))
    
    fig.update_layout(
        title=f"{ip_version} VPN Transitions from 2025-06-13 to 2025-09-23",
        height=1000,
        margin=dict(t=30, b=10, l=10, r=10)
    )
    
    fig.show()
    
    # Quick summary
    total = data['count'].sum()
    stable = data[data['service_old'] == data['service_new']]['count'].sum()
    print(f"{total:,} networks, {stable/total*100:.1f}% stable")

print("Sankey function ready!")


Sankey function ready!


In [15]:
# Analyze IPv4 VPN service transitions
ipv4_transitions = analyze_vpn_service_transitions(df_ipv4_20250613, df_ipv4_20250923, "IPv4")


=== IPv4 VPN Service Provider Transitions ===
VPN networks in 2025-06-13: 10,719,408
VPN networks in 2025-09-23: 11,043,477
Common VPN networks: 5,302,295

Total service transitions tracked: 362
Top 10 transitions:
               service_old            service_new    count
240    Unknown/Generic VPN    Unknown/Generic VPN  5252375
298                VPNGate                VPNGate     6097
56                 FastVPN                FastVPN     5849
127                NordVPN                NordVPN     4195
98                IPVanish               IPVanish     2437
183           Troywell VPN           Troywell VPN     2307
191             TunnelBear             TunnelBear     2019
147              ProtonVPN              ProtonVPN     1989
142  PrivateInternetAccess  PrivateInternetAccess     1780
172              Surfshark              Surfshark     1527


In [41]:
# Remove Unknown/Generic VPN to Unknown/Generic VPN transition if it exists
ipv4_transitions = ipv4_transitions[~((ipv4_transitions['service_old'] == 'Unknown/Generic VPN') & 
                                     (ipv4_transitions['service_new'] == 'Unknown/Generic VPN'))]


ipv4_transitions = ipv4_transitions.sort_values(by='count', ascending=False)
print(ipv4_transitions.head(100))

# Create Sankey for IPv4
if ipv4_transitions is not None:
    create_vpn_service_sankey(ipv4_transitions, "IPv4", min_count=200)


      service_old   service_new  count
298       VPNGate       VPNGate   6097
56        FastVPN       FastVPN   5849
127       NordVPN       NordVPN   4195
98       IPVanish      IPVanish   2437
183  Troywell VPN  Troywell VPN   2307
..            ...           ...    ...
344     myiphider     myiphider     53
123        Njalla        Njalla     52
24       BelkaVPN      BelkaVPN     49
306     VPNTunnel     VPNTunnel     48
84      HideIPVPN     HideIPVPN     47

[100 rows x 3 columns]
Total transitions before filtering: 361
IPv4: 37 transitions after filtering


41,107 networks, 88.4% stable


In [43]:
# Analyze IPv6 VPN service transitions
ipv6_transitions = analyze_vpn_service_transitions(df_ipv6_20250613, df_ipv6_20250923, "IPv6")

# Create Sankey for IPv6
if ipv6_transitions is not None:
    create_vpn_service_sankey(ipv6_transitions, "IPv6", min_count=1)  # Lower threshold for IPv6



=== IPv6 VPN Service Provider Transitions ===
VPN networks in 2025-06-13: 1,124
VPN networks in 2025-09-23: 1,507
Common VPN networks: 889

Total service transitions tracked: 20
Top 10 transitions:
            service_old          service_new  count
10              Mullvad              Mullvad    506
12                 OVPN                 OVPN    229
2               Anonine                BoxPN     42
8               MaxiVPN              MaxiVPN     34
11               Njalla               Njalla     29
0                  1VPN                 1VPN     11
9               MaxiVPN                v2vpn      9
15         Troywell VPN         Troywell VPN      7
7                  Jego                 Jego      4
16  Unknown/Generic VPN  Unknown/Generic VPN      4
Total transitions before filtering: 20
IPv6: 20 transitions after filtering


889 networks, 93.9% stable


In [44]:
# Create summary analysis of the most significant service provider changes
def analyze_top_transitions(transitions_df, ip_version, top_n=10):
    """Analyze the top service provider transitions"""
    
    if transitions_df is None or len(transitions_df) == 0:
        return
    
    print(f"\n=== TOP {top_n} {ip_version} VPN SERVICE TRANSITIONS ===")
    
    # Sort by count and get top transitions
    top_transitions = transitions_df.sort_values('count', ascending=False).head(top_n)
    
    print(f"{'Rank':<4} {'From Service':<25} {'To Service':<25} {'Networks':<10} {'Type'}")
    print("-" * 80)
    
    for i, (_, row) in enumerate(top_transitions.iterrows(), 1):
        from_service = row['service_old'][:24] if len(row['service_old']) > 24 else row['service_old']
        to_service = row['service_new'][:24] if len(row['service_new']) > 24 else row['service_new']
        
        if row['service_old'] == row['service_new']:
            transition_type = "STABLE"
        else:
            transition_type = "CHANGE"
            
        print(f"{i:<4} {from_service:<25} {to_service:<25} {row['count']:<10,} {transition_type}")
    
    # Calculate stability metrics
    total_networks = transitions_df['count'].sum()
    stable_networks = transitions_df[transitions_df['service_old'] == transitions_df['service_new']]['count'].sum()
    changed_networks = total_networks - stable_networks
    
    print(f"\n{ip_version} VPN Service Stability Summary:")
    print(f"Total networks tracked: {total_networks:,}")
    print(f"Stable (same service): {stable_networks:,} ({stable_networks/total_networks*100:.1f}%)")
    print(f"Changed service: {changed_networks:,} ({changed_networks/total_networks*100:.1f}%)")
    
    # Top gaining and losing services
    print(f"\nNet Changes Analysis:")
    
    # Calculate net gains/losses for each service
    service_changes = {}
    
    for service in transitions_df['service_old'].unique():
        lost = transitions_df[(transitions_df['service_old'] == service) & 
                            (transitions_df['service_new'] != service)]['count'].sum()
        gained = transitions_df[(transitions_df['service_new'] == service) & 
                              (transitions_df['service_old'] != service)]['count'].sum()
        net_change = gained - lost
        service_changes[service] = net_change
    
    # Show top gainers and losers
    sorted_changes = sorted(service_changes.items(), key=lambda x: x[1], reverse=True)
    
    print("Top 5 Service Gainers:")
    for service, change in sorted_changes[:5]:
        if change > 0:
            print(f"  {service}: +{change:,} networks")
    
    print("Top 5 Service Losers:")
    for service, change in sorted_changes[-5:]:
        if change < 0:
            print(f"  {service}: {change:,} networks")

# Analyze transitions for both IP versions
if ipv4_transitions is not None:
    analyze_top_transitions(ipv4_transitions, "IPv4")

if ipv6_transitions is not None:
    analyze_top_transitions(ipv6_transitions, "IPv6")



=== TOP 10 IPv4 VPN SERVICE TRANSITIONS ===
Rank From Service              To Service                Networks   Type
--------------------------------------------------------------------------------
1    VPNGate                   VPNGate                   6,097      STABLE
2    FastVPN                   FastVPN                   5,849      STABLE
3    NordVPN                   NordVPN                   4,195      STABLE
4    IPVanish                  IPVanish                  2,437      STABLE
5    Troywell VPN              Troywell VPN              2,307      STABLE
6    TunnelBear                TunnelBear                2,019      STABLE
7    ProtonVPN                 ProtonVPN                 1,989      STABLE
8    PrivateInternetAccess     PrivateInternetAccess     1,780      STABLE
9    Surfshark                 Surfshark                 1,527      STABLE
10   CyberGhost                CyberGhost                1,145      STABLE

IPv4 VPN Service Stability Summary:
Total networks